In [1]:
import pandas as pd

In [2]:
file_endings = "GSE207998"
normal_count = 94

In [3]:
train = pd.read_csv(f"../champ_result/all_beta_normalized_{file_endings}.csv")

In [4]:
# id_list = pd.read_csv("../result/id_list.txt", header=None)

In [5]:
# train = train[train["Unnamed: 0"].isin(id_list[0])]

In [6]:
train_normal = train.iloc[:, 1:normal_count+1:2].T
train_tumor = train.iloc[:,normal_count+1::2].T

##### Calculate Δβ

In [7]:
# remove outlier based on every column
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + IQR*1.5
    lower_fence = Q1 - IQR*1.5
    return upper_fence,lower_fence
def no_outlier(df):
    upper_fence, lower_fence = IQR(df)
    ddf=df[(df>lower_fence)&(df<upper_fence)]
    return ddf

In [8]:
train_normal = no_outlier(train_normal)
train_tumor = no_outlier(train_tumor)

In [9]:
train_normal_avg = train_normal.mean(skipna=True, axis = 0)

In [10]:
train_tumor=(train_tumor).subtract(train_normal_avg, axis = 1)

In [11]:
train_tumor=no_outlier(train_tumor)

In [12]:
train_tumor_mean = train_tumor.mean(skipna=True, axis = 0)

In [13]:
Δβ = pd.merge(train.iloc[:,:1], pd.DataFrame(train_tumor_mean, columns = ["dbeta"]), left_index=True, right_index=True)
Δβ

,Unnamed: 0,dbeta
0,cg00000957,0.005332
1,cg00001349,0.024420
2,cg00001583,0.279669
3,cg00002028,0.050706
4,cg00002719,0.361206
...,...,...
404849,cg27656573,-0.001472
404850,cg27657363,-0.137955
404851,cg27657537,-0.007020
404852,cg27662611,-0.003883


In [14]:
dmp_train = pd.read_csv(f"../champ_result/DMP_result_{file_endings}.csv")
print(f"raw train shape: {dmp_train.shape}")
dmp_train = dmp_train[["Unnamed: 0", "gene"]]
dmp_train.dropna(inplace=True)
print(f"after dropna: {dmp_train.shape}")

raw train shape: (347250, 24)
after dropna: (267235, 2)


In [15]:
result = pd.merge(Δβ, dmp_train, on="Unnamed: 0", how="inner")

In [16]:
def find_max_dBeta_grouped(group):
    idx_max = group['dbeta'].abs().idxmax()
    return group.loc[idx_max]

result_max_per_gene = result.groupby("gene").apply(find_max_dBeta_grouped).reset_index(drop=True)

In [17]:
single = pd.read_csv('../comorbidity/matchgene174_single_3Y10__OR2.txt', sep='\t', header=None)

In [18]:
result_max_per_gene_single = result_max_per_gene[result_max_per_gene['gene'].isin(single[0])]
result_max_per_gene_single

,Unnamed: 0,dbeta,gene
1,cg10222734,-0.389851,A1CF
4,cg08300930,0.246998,A2M
5,cg23546356,-0.340394,A2ML1
12,cg13001012,-0.138395,AADAC
16,cg18716979,0.081409,AADAT
...,...,...,...
18936,cg07468289,-0.383186,ZNRD1
18939,cg20080983,-0.216836,ZNRF3
18956,cg14231297,0.432943,ZSCAN18
18976,cg14642833,-0.015741,ZWINT


In [19]:
result_max_per_gene_single.to_csv(f"../result/result_max_per_gene_single_{file_endings}.csv", index=False)

In [26]:
check = result_max_per_gene_single.loc[result_max_per_gene_single["dbeta"].abs() > 0.35]
print(check.shape)
check

(960, 3)


,Unnamed: 0,dbeta,gene
1,cg10222734,-0.389851,A1CF
58,cg14592673,-0.391966,ABCC3
59,cg18016288,-0.363512,ABCC4
103,cg11149849,-0.393696,ABR
129,cg00507135,-0.389435,ACBD5
...,...,...,...
18540,cg08272268,-0.366877,ZNF281
18553,cg02343823,0.362446,ZNF300
18620,cg07633317,-0.439770,ZNF423
18936,cg07468289,-0.383186,ZNRD1
